In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from hand_analysis import process_poker_hand, save_to_csv
from poker_app.pypokergui.engine.card import processed_hand, percentiles_table

preflop_action_processed
0    0.586213
1    0.413787
Name: proportion, dtype: float64
Probabilities: [[0.36 0.64]]
Decision: raise
Accuracy: 0.73
Rozmiar y_train: (4816,)
Rozmiar X_test: (1204, 2)
Rozmiar y_test: (1204,)
Sensitivity: 0.8329938900203666
Specificity: 0.6591865357643759


In [2]:
# Mapowanie pozycji i akcji
position_map = {'EP': 0, 'MP': 1, 'CO': 2, 'BTN': 3, 'SB': 4, 'BB': 5}
action_map = {'F': 0, 'R': 1}
FILES_PATH = 'hh/*.txt'
hands = process_poker_hand(FILES_PATH)
save_to_csv(hands)

In [3]:
df = pd.read_csv('poker_hand.csv')
df = df[~df['preflop_action'].isin(['X', 'C', 'B'])]  # Usuwamy niechciane akcje
df['position_processed'] = df['position'].map(position_map)  # Mapowanie pozycji
df = df.dropna(subset=['position_processed'])  # Usuwanie wierszy z brakującymi pozycjami
df['hole_cards'] = df['hole_cards'].apply(lambda x: eval(x) if isinstance(x, str) else x)  # Przetwarzanie kart
df['preflop_action_processed'] = df['preflop_action'].map(action_map)
df['preflop_action_processed'] = df['preflop_action_processed'].fillna(0) 



In [7]:
sample_hand = [('Ts', '9s'), 'BTN']
hand_strength = processed_hand(sample_hand[0], percentiles_table)
# df_filtered = df[
#     df['hole_cards'].apply(lambda x: processed_hand(x, percentage_table) <= hand_strength)
#     ]
X_filtered = pd.DataFrame({
    'hand_strength': df['hole_cards'].apply(lambda hand: processed_hand(hand, percentiles_table)),
    'position': df['position_processed']
})

y_filtered = df['preflop_action_processed']
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
sample_hand_processed = pd.DataFrame(
    [
        [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [6, 3], [7, 3], [8, 3], [9, 3], [10, 3],
        [11, 3], [12, 3], [13, 3], [14, 3], [15, 3], [16, 3], [17, 3], [18, 3], [19, 3], [20, 3],
        [21, 3], [22, 3], [23, 3], [24, 3], [25, 3], [26, 3], [27, 3], [28, 3], [29, 3], [30, 3],
        [31, 3], [32, 3], [33, 3], [34, 3], [35, 3], [36, 3], [37, 3], [38, 3], [39, 3], [40, 3],
        [41, 3], [42, 3], [43, 3], [44, 3], [45, 3], [46, 3], [47, 3], [48, 3], [49, 3], [50, 3],
        [51, 3], [52, 3], [53, 3], [54, 3], [55, 3], [56, 3], [57, 3], [58, 3], [59, 3], [60, 3],
        [61, 3], [62, 3], [63, 3], [64, 3], [65, 3], [66, 3], [67, 3], [68, 3], [69, 3], [70, 3],
        [71, 3], [72, 3], [73, 3], [74, 3], [75, 3], [76, 3], [77, 3], [78, 3], [79, 3], [80, 3],
        [81, 3], [82, 3], [83, 3], [84, 3], [85, 3], [86, 3], [87, 3], [88, 3], [89, 3], [90, 3],
        [91, 3], [92, 3], [93, 3], [94, 3], [95, 3], [96, 3], [97, 3], [98, 3], [99, 3], [100, 3],
        ],
    columns=['hand_strength', 'position'] 
)
probabilities = np.round(model.predict_proba(sample_hand_processed), 2)
decision = model.predict(sample_hand_processed)
# scores = cross_val_score(model, X_filtered, y_filtered, cv=5, scoring='accuracy')

In [5]:
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
